In [1]:
# 앞절에서 수행한 내용들

import os
import numpy as np
from sklearn.datasets import fetch_openml

dataset_root = os.path.join(os.getcwd(), 'datasets')
if not os.path.isdir(dataset_root):
    os.mkdir(dataset_root)
    
mnist = fetch_openml('mnist_784', version=1, data_home=dataset_root)
X, y = mnist["data"], mnist["target"].astype(np.uint8)

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

some_digit = X_train[10]
some_digit_answer = y_train[10]

# Multilabel Classification

지금까지는 하나의 instance에 하나의 class만이 지정된 경우를 살펴보았지만, 한 instance에 여러 class를 지정해야 하는 경우가 있을 수 있다.

예를 들어, face-recognizer가 Alice, Bob, Charlie의 3개의 class를 인식하도록 학습되어, 한 이미지에서 \[1, 0, 1\]로 output을 하는 것이다.(사진에 Alice와 Charlie만 있는 경우)

위와 같이 여러 binary tag를 출력하는 classification system을 **multilabel classification** system이라고 한다.

간단한 예를 들어보자.

In [2]:
from sklearn.neighbors import KNeighborsClassifier

y_train_large = (y_train >= 7)
y_train_odd = (y_train % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

`y_multilabel` array는 2개의 target label을 가진다.(7보다 같거나 큰지?, 홀수 인지?) 그리고 multilabel classification을 지원하는 `KNeighborClassifier`를 사용해 학습하였다.

이제 prediction을 통해 output의 label이 어떻게 나오는지 확인해보자.

In [3]:
print(some_digit_answer)
print(knn_clf.predict([some_digit]))

3
[[False  True]]


많은 multilabel classification의 평가 방법이 있고, 올바른 성능 척도를 선택하는 것은 프로젝트에 따라 다르다. 예를 들어, 한가지 방법으로 각 label에 대해 $F_1$ score(또는 앞절에서 다룬 binary classifier의 다른 척도)를 측정한 후, 평균 점수를 내는 것이 있다.

다음은 모든 label에 대한 $F_1$ score를 계산해 평균내는 것이다. 

In [4]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)
print(f1_score(y_multilabel, y_train_knn_pred, average='macro'))

0.976410265560605


위 코드는 모든 label이 모두 중요하다고 가정한다. (`average='macro'`)

만약, 각 label의 instance수가 차이가 나는 경우에는 해당 instance의 수 비율과 같도록 weight를 부여할 수도 있다. 이는 간단히 `average='weighted'`로 설정해주면 된다.